In [7]:
class Field:
    def __init__(self, size, item_pickup, item_dropoff, start_position):
        self.size = size
        self.item_pickup = item_pickup
        self.item_dropoff = item_dropoff
        self.position = start_position
        self.item_in_car = False
        
    def get_state(self):
        state = self.position[0]*self.size*self.size*self.size*2
        state = state + self.position[1]*self.size*self.size*2
        state = state + self.item_pickup[0]*self.size*2
        state = state + self.item_pickup[1]*2
        
        if self.item_in_car:
            state = state + 1
        return state
        
    def get_number_of_states(self):
        return self.size*self.size*self.size*self.size*2

    def make_action(self, action):
        (x,y) = self.position
        if action == 0: #down
            if y == self.size-1:
                return -10, False
            else:
                self.position = (x, y+1)
                return -1, False
            
        elif action == 1: #up
            if y == 0:
                return -10, False
            else:
                self.position = (x, y-1)
                return -1, False
            
        elif action == 2: #left
            if x == 0:
                return -10, False
            else:
                self.position = (x-1, y)
                return -1, False
                
        elif action == 3: #right
            if x == self.size-1:
                return -10, False
            else:
                self.position = (x+1, y)
                return -1, False
            
        elif action == 4: # pickup
            if self.item_in_car:
                return -10, False
            elif self.item_pickup != (x,y):
                return -10, False
            else:
                self.item_in_car = True
                return 20, False
                
        elif action == 5: #dropoff
            if not self.item_in_car:
                return -10, False
            elif self.item_dropoff != (x,y):
                self.item_pickup = (x,y)
                self.item_in_car = False
                return -10, False
            else:
                self.item_in_car = False
                return 20, True


In [67]:
size=10
item_pickup=(0,0)
item_dropoff=(9,9)
start_position=(7,0)
field = Field(size, item_pickup, item_dropoff, start_position)

In [64]:
field.position

(8, 0)

In [68]:
field.make_action(2)
field.make_action(2)
field.make_action(2)
field.make_action(2)
field.make_action(2)
field.make_action(2)
field.make_action(2)
field.make_action(2)
field.make_action(2)

(-10, False)

In [69]:
field.position

(0, 0)

In [70]:
field.make_action(4)

(20, False)

In [71]:
field.make_action(0)
field.make_action(0)
field.make_action(0)
field.make_action(0)
field.make_action(0)
field.make_action(0)
field.make_action(0)
field.make_action(0)
field.make_action(0)


(-1, False)

In [73]:
field.make_action(3)
field.make_action(3)
field.make_action(3)
field.make_action(3)
field.make_action(3)
field.make_action(3)
field.make_action(3)
field.make_action(3)
field.make_action(3)

(-10, False)

In [74]:
field.position


(9, 9)

In [75]:
field.make_action(5)

(20, True)

In [3]:
# Random Solution
import random
def random_solution():
    size = 10
    item_pickup = (0,0)
    item_dropoff = (9,9)
    start_position = (9,0)

    field = Field(size, item_pickup, item_dropoff, start_position)
    
    done = False
    steps = 0
    
    while not done:
        action = random.randint(0,5)
        reward, done = field.make_action(action)
        steps = steps + 1
        
    return steps
    

In [80]:
random_solution()

181676

In [8]:
import numpy as np

#q learning Algo 
size = 10
item_pickup = (0,0)
item_dropoff = (9,9)
start_position = (9,0)
field = Field(size, item_pickup, item_dropoff, start_position)
number_of_states = field.get_number_of_states()
number_of_actions = 6



q_table = np.zeros((number_of_states, number_of_actions))

epsilon = 0.1
alpha = 0.1
gamma = 0.6




for _ in range(10000):
    field = Field(size, item_pickup, item_dropoff, start_position)
    done = False
    
    while not done:
        state = field.get_state()
        if random.uniform(0,1) < epsilon:
            action = random.randint(0,5) #Explore
        else:
            action = np.argmax(q_table[state]) #Exploit
            
        reward, done = field.make_action(action)
        
        new_state = field.get_state()
        new_state_max = np.max(q_table[new_state])
        
        q_table[state, action] = (1-alpha)*q_table[state, action]+alpha*(reward+gamma*new_state_max - q_table[state, action])
                        

In [9]:
q_table

array([[ 0.23071429, -2.06428571, -2.06428571,  0.23071429,  9.78571429,
        -2.06428571],
       [-0.71428571, -5.21428571, -5.21428571, -0.71428571, -5.21428571,
        -2.06428571],
       [ 1.13876935, -1.        , -1.        , -0.1       ,  0.        ,
         0.        ],
       ...,
       [-1.8       , -0.1       , -0.306     , -1.47294337, -1.75883129,
        10.10310751],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [10]:
def reinforcement_solution():
    epsilon = 0.1
    alpha = 0.1
    gamma = 0.6
    
    field = Field(size, item_pickup, item_dropoff, start_position)
    done = False
    steps = 0
    
    while not done:
        state = field.get_state()
        if random.uniform(0,1) < epsilon:
            action = random.randint(0,5) #Explore
        else:
            action = np.argmax(q_table[state]) #Exploit
            
        reward, done = field.make_action(action)
        
        new_state = field.get_state()
        new_state_max = np.max(q_table[new_state])
        
        q_table[state, action] = (1-alpha)*q_table[state, action]+alpha*(reward+gamma*new_state_max - q_table[state, action])
                        
        steps = steps +1
        
    return steps

In [11]:
reinforcement_solution()


30

In [12]:
run = [reinforcement_solution() for _ in range(100)]

In [13]:
sum(run)/len(run)

41.49